# 재귀 신경망, 합성곱 신경망

In [30]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import Adam
import numpy as np
import random, sys
import logging
logging.disable(logging.WARNING)
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_datat_data

### 재귀 신경망

In [13]:
fp = codecs.open('/Users/lesson6_mac/AtomData/python/words/class/7CM00009.txt', 'r', encoding='utf-16')
soup = BeautifulSoup(fp, "html.parser")
text = soup.getText() + " "

In [16]:
print('말뭉치의 길이: ', len(text))
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수: ', len(chars))

말뭉치의 길이:  49722
사용되고 있는 문자의 수:  865


In [17]:
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i, c in enumerate(chars))
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수: ', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')

학습할 구문의 수:  16568
텍스트를 ID 벡터로 변환합니다...


In [18]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('모델을 구축합니다...')

모델을 구축합니다...


In [19]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [20]:
def sample(preds, temperature= 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [22]:
for iteration in range(1,50):
    print()
    print('-' *50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ""
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.0
            # 다음에 올 문자르 예측
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 문장을 출력
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
반복 = 1
Epoch 1/1
16568/16568 [==============================] - 10s 626us/step - loss: 4.5247

--- 다양성 =  0.2
--- 시드 = " 눌러 보자.
이게 아닌데?
영점 오"
 눌러 보자.
이게 아닌데?
영점 오
     
   

   
  
 
               ,
    
  .  
  
       


   
   
 
                
                  
     
 ,  
 

          

  
 
 
  
    
  
                     
          
 
        

       
    
            ,        
            
                 
     
     
  
       
           
    
  
      
                                        
   
                    
              
 
    

--- 다양성 =  0.5
--- 시드 = " 눌러 보자.
이게 아닌데?
영점 오"
 눌러 보자.
이게 아닌데?
영점 오.



    
?  

 ,막  서

 
 

 
  
 .
  :  

이이니  

 . 다     .    


  

   
 
 ,
?  근런 
  
  
, 
 ,
가  ,
  

.   
은  도   
고 .
 
  
  데 는          
  고 
 .:    
 .   ?
 .     

 
  요  ,
.
 어
 
   
.
   지  
  
이 이 

 지
 
 


  
 
 아 서에  ,    
., 
 이 안 그
,    ,. 
    고, .

이
  
 요   

 

아

 
 
 .는   지이 
 .


을 
 고
,   

16568/16568 [==============================] - 11s 658us/step - loss: 3.9435

--- 다양성 =  0.2
--- 시드 = "지고.
걔가 관광 쪽 갈라고,
아.
"
지고.
걔가 관광 쪽 갈라고,
아.
그  이  가  어 
그  이 이 
.
그  이 
 이 이  이 이 
근  이 
이  이 
이  이 
이 
그  이  이  이  이  그  이  이 이 이 이 이  이 
 어 이  이  이 
 이 
이 
     이 이  이 
이  이  이 
 아  이 이 이  이  이  이  이  이  이 
이  이  이  데 
이  이  이  있 이  이 이 .
그 ,
그   가 
이  이  어 
그  그 이  이  이 
이  이  아 이 거  이 
이  이 
이 이  이 이  이  아 
이 
이  이 ,
그  이  이 
 이  이 
 이  이  어 
그     .
그  이 .
그  하  이  이 이 이 이 
이  이 이 이  이 
이  지 
이       이 이 아 
이 .
그  이 
이  아  이  이 이  그  이

--- 다양성 =  0.5
--- 시드 = "지고.
걔가 관광 쪽 갈라고,
아.
"
지고.
걔가 관광 쪽 갈라고,
아.
그가 

 게  어.
어 ,
그  에 를 
어 
기 런 다,

근 이 .
그  이 어 그 그어 
그   이 이 보  러 차 이  거 니 .
그.
그  다 다.
그 어.
.
이 아 안 , 있 
그 나 
도 안 그지 ,
그.
그 는  지 ,
그  아  그.
그 같  가 그 .
아 
대  이: 그면  하.
이  이 
이 거  .
아  그이 
기 이 나 ,
그러  이 뭐  스 

이 이 요,
그,

어 자.
그는 
 이 지 아 
그 : 
이 
거 이  아 하  막 니 
이 
니 대  다.
그 한 이 이 이   는 
이 명 안 그 대 야 이 
 이 
어  .
거 칠 그 ,
이 게 이,
그 근 가 가 그   나 
이 이 가 안 
이 데 놈나 ?
그 .
이 
.
상는 
무 
나  이?
어마  기

정규 분포가 그거 이 이 이 이 이 이 이 이 이 이 있 이 거 이 거 거 이 이 있 이 이 이 이 거 이 이 이 이 거 이 이 거 이 이 이 이 이 이 이 이 이 이 이 하 거 거 이 하 거 이 이 거 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 이 있 이 다 하는 이 이 이 이 거 이 이 이 이 이 있 이 거 이 이 거 거 이 이 이 이 이 이 하고 그 이 이 이 이 이 이 이 이 이 있는 말 이 이 이 이 이 거 이 이 이 이 하 이 이 이 이 이 이 이 이 거 이 이 이 이 이 이 있 거 이 이 이 이 이 이 가 있 이 있 이 이 이 거 이 이 이 이 이 이 이 이 하고 이 이 이 아 이 이 이 이 이 이 이 거 거 있 이 이 이 이 이 이 이 이 이 이 거 이 이 이 하 이

--- 다양성 =  0.5
--- 시드 = "발견했다,
놀라운,
정규 분포가 그거"
발견했다,
놀라운,
정규 분포가 그거 나가 이 같 거 고 이 그 나 가 이 같어면요.
막.
이고 거는 안 이 이 나는 안 내 이 나.
그렇 말요.
이 하대 있는 가 하기 하은 고 평어.
그 하 구 거 좋라.
그렇가 같 이 와.
아.
그리 이 안 요.
아.
그러 거 이 때 많이 이 같 있.
이이고,
그런 거 이 다  야 거 ,
그 대 같는 되이 이 수 생이는 거 구리 아 거 그 하기 시 ?
어.
그지 되지.
아.
응,
그 데 
어.
아.
뭐.
그런 이 같가 나 히 해고 이제 의 분 같차 이 가 ,


이고 되고 편학면 면 수 다 가 으 이 한 있 같다 하지 되상이 거 있래 거 않.
그는 보 한 아.
이 아 거 단이 아부 있이,
이.
그 거 이 ,
이런 거 뭐~ 뭐~,
그트 거 이가 거  한 해 ?
지.
그 
아아,
그워.
그런 거 이 하 있이 

--- 다양성 =  1.0
--- 시드 = "발견했다,
놀라운,
정규 분포가 그거"
발견했다,
놀라운,
정규 분포가 그거 좋학야 알단방,
사죠히의 이박이들.
예.
생래 중나고 됩바착슨하거거지.
이케 평이에가라을:에러

그런 거 있는 거 있는 거 거 있는 거 하고 거 있고 그런 거 이 같은 거 있어.
그런 거 같은 거 하는 그런 거 있는 거 있어요.
그런 거 나지.
그래 거 있어.
그런 거 이 이 있는 거 이 있는 거 같고 거 있고 거 있어.
그래 거 이 다 이 거 있는 거 있어요.
그런 거 있어.
그래 이 하고 거 있고 거 이 이 이 이 나 거 거 거 다.
그래 거 아.
그렇게 거 하고 거 있어.
그런 거 있고 거 아.
그래 거 아.
그런 거 이 같은 거 거 이 다 이 지.
그런 거 이 이 하고 거 있어.
그런 거 있는 거 거 이 다 같은 거 같은 거 있어.
그런 거 있는 거 같은 거 하는 거 이 아.
그런

--- 다양성 =  0.5
--- 시드 = "래서 이거 더하기,
이거 더하기를 하"
래서 이거 더하기,
이거 더하기를 하지 하고 되다,
아.
그런 거 있 무 지.
그래 다 같는 들어?
그런 다.
성~ 그런 얘 거 같잖아.
그래 평생 거 하고 안 그니까,
그런 이게 막~ 거 오지.
그래서,
어서 뭐~ 싶면 되으서 이렇게 거 같고 그런 거고,
근데 지~ 그런 거 많고 없이 다.
그런 그런 거 이는 그렇게 하가 거 없 아.
생리 이 서 오거 다.
그래서,
그런 뭐~ 말 있는 이 해으 거 말 만 딱 일 다 다.
아.

응.
그렇지 그 가 시 한 하지 가 거 거 없고.
얘 이 한 고 이 있 아겠는 이 이 보면,
아.
그런 거 쪽고 거 들어.
이래 이 버 이 이러 나거 아.
그래서,
그렇게 이렇게 있지 다 있고 그런 거 없어.
그런 수 같 지 있지 다지?
그런 막~ 구서 아요.
하가 거 있는데,



아.
그런 이 같는 뭐~ 한 있시,

--- 다양성 =  1.0
--- 시드 = "래서 이거 더하기,
이거 더하기를 하"
래서 이거 더하기,
이거 더하기를 하쌩,
그본가 지격반.나십 같잖 야.
수무에.
나~ 얼지면,
갱말 하기다 그래 달 젝신서 싶으가 있는데 얘은 안 못 좋,
그러로 꿈어일?
응.
으아.
어.
게 ~ 쪽과 계들,
막렇을?
그럼 삼샤 서 즉 든터?
아사가.
그런 많도 

그런 거 같은 거 같은 거 같은 거 같은 거 있고 거 있고 거 하고 거 있고 거 같은 거 있어요.
그런 거 같은,
그러면,
아.
그러고 그런 거 같은 거 있는데,
그런 거 있고 거 같은 거 있는데,
그래서 거 있고 거 같은 거 다.
그래서 있어요.
그렇게 거 있는데,
그래서 이 같은 거 같은 거 같은 거 같은 거 같은 거 있는데,
그런 거 있고,
그런 거 같은 거 같은 그런 거 있어.
그런 거 있는데,
그렇게 거 있는데,
그래서 

--- 다양성 =  0.5
--- 시드 = "규 분포를 따를 수 있다고,
얘기할 "
규 분포를 따를 수 있다고,
얘기할 가 까?
진짜 되라 다기 했어요.
이 안 한 이 있고,
그니까,
그렇게 거.
그래서,
이 없이 거 같은 이렇게 있어요.
그래서,
지 이리 소 나을 거 다 같은 거 하는 같은 막~,
이 이 하면,
그런 나고,
일 해 이 을 거 있지.
그런 있는 거 거 있고 거 하고 있어.



어요.
그래서 다.
응.
그런 거 같은 거 지.
그래서 이렇지?
그 마인 그런 거 되겠지?
그 같은 거 같은 거 나람 거 지면,
그러고,
학 들이 거 있고 거 같는 거 지.
그러면 막~,
이니까 거 같은,
이 하는 평균이 어요?
그게 또 있잖아요,
그러면요,
그 같은 애들 나고 거 이면 거,
그 게 나는 있는 거.
그러면 거 같은 거데,
응.
그렇게 거 다고 일 보면 거는 거 다가 거 거고,
그냥 그런 뭐~ 거 하고 될 오면,
그런 거

--- 다양성 =  1.0
--- 시드 = "규 분포를 따를 수 있다고,
얘기할 "
규 분포를 따를 수 있다고,
얘기할 아 많자고고.
나.
뭐은 뭐~ 좀,
모집
딱.
그렇잖아요.
어.
네게 되고?
빠면 .
표기한 장메상 있는 같은요,
위또 원가지 방니어.
방X나었
하게 하는요?예이 다리 둬목 봐장.
쫌 하고 막~ 막~,

0에 엠서은 생교심한 진짜,
근데 다격.
예 어은 거까요,
카의 세 입으가 오렇야고,
근데,
내직에 파재었어가 여면 말른대,
엄어 가는 확고 수박 거았고,
에이 요 먹면비,
불츠지 어규 그런

...

그래서 생이십다 있어요.
그러면은,
그래서 되게 거 같애요.
그래서 이렇게 많이 있는데,
이렇게 같은 거 같애요,
일 다가 있어요.
응.
그게 막~ 얘가 하다 있어요.
그마?
아 이런 것도 있잖아.
그래서 이렇게 뭐~ 모집단은 사람들 있지?
이렇게 좋아요?
그 다음 되게 되잖아.
그런 거,
그런 거는.
그 같은 거.
그지?
그 다음에 제 그래서 너무 이거.


그래서 막~ 막~ 막~ 이런 거 있는 거 같애요,
그래서 그렇지,
너무 막~ 막~ 이렇게 되고 되는 거야.
그래서 그래서 이렇게 되게 게 하면,
얘가 그런 거 같애요.
그 같은 거 같은,
이렇

--- 다양성 =  1.0
--- 시드 = "
네.
얘는 십의 제곱이었고,
얘는 "

네.
얘는 십의 제곱이었고,
얘는 매려 깐한 연네 그리고 좋은 돼 게,
보고,
선분에 읽어요.
막~,
복학하기 오지 들이야?
어쪽마 필성구,
찬간 체 못 오신 있는 거 하자까?
가.
음 지더라.
왜 공거 다 봐망구,
그래에,
자:: 가지 그러까거요.
얘가는 공음이 따돼 당혀면 될 게 있는데,
고,
응.
막가,
되게 버어,
당 수의 가 먹고 장르면 막~ 그냥 안 좋아요?
이,
디 불 읽어 한내가


.
에희보


그래서 으나,
어떻게 나?
정규 그래 살구하대,

이가 이렇게 나킨지니까.
예져 크앙는 거 같은,
미연서 딱 타균었고,
응?
막~,
통끔 이렇게,

이기과 되게 많고 뭉때,
그때 전으로 임흔 데.
중유?
이니의 계복이 충가지구오 오 아녔::::,
그때 나~ 좋은 좀,
걔.
그래서 이런 허때 영느에,
얘 꼭 이다 ,
그런 쳤에다고.
통

--- 다양성 =  1.2
--- 시드 = "
네.
얘는 십의 제곱이었고,
얘는 "

네.
얘는 십의 제곱이었고,
얘는 표케 작들으로 안 처는데 모르고 받고 있으면져요?
어일보누 달
김X 기가 이가서하는 싶응.
그래?
띡.
닮생나 안 같애요,
그어.
교규 수타도 벌다는 거 되.
오과일국 과시라간가,

백에서요게 빠학 한만이 무덩더,
복준 독트처 안 유상원외다무를 집십도 거화하고?
잔합관데서

어.
그러니까.
되게 그런 거 같은 거 같애요.
그냥 제가 이런 거 있어요.
그런 거 같애요.
그렇게 아니고,
그 다음 그런 거 같애요.
그런 거 같애요.
그냥 거 같은 거 같애들.
너무,
그래서 막~ 이런 거 같애요.
그래서 그런 거 있어요.
그런 거 같은데,
그런 거 같애요.
어떻게 그런 거 아니지?
그러면,
그런 애들 하면 거지.

그지?
이렇게 있어요.
이거 상이 싫어

--- 다양성 =  1.0
--- 시드 = "뭐야,
살아남을 수 있는 게,
다 그"
뭐야,
살아남을 수 있는 게,
다 그랬어서 아학도 일도 되는데,
오:: 버거 같은,
그걸 사대요?
그게.
한라고 구날 이케에,
막~ 이렇게 일 때 헤사못이 거예요?
진짜 그런 초?
이형::,
근데 감형 싶죠.
이렇게 나타,
그기, 걔 안 닦어 쪽은하지,
선동들은 얘 돼고 있나면.
그렇지 가지가?
막~ 비아형어,
언생 게 하지 나매요?
이,
얘가 이런 그런 나고 좀,
확를 갈 때.
이즘,
아우,
이.
그냥 있겠지니,
난 한 돌드왔고,
이상 뭔르 말생야.
보면 안 나형이삐이~,
네.
어 내각하인 들원으든까는데지.
요긴에니었XX,
불연대야.
응.
이것을 것도 가 보고,

그럼 하서 과만해 게 있는데,
아.
글기 들어서 데라 야예요?
응.
그런 거야.
응.
진짜 따래 가면,
드신 된간를 주엑스가 생거예요.
어떻게 게,
않지?
근데 근데 그래서.
찍에

--- 다양성 =  1.2
--- 시드 = "뭐야,
살아남을 수 있는 게,
다 그"
뭐야,
살아남을 수 있는 게,
다 그와?
이것.
이유?
어,
편정도 폴안 좋으네.
언어,
작트 질지거,
또 불거지.
아,
스간도.
쪼놈들이 왔럽,
건예고도.
맞어.
그마죠.
응.
요영토도,
정규 함얀검은 부~ 이라고 노섭요,
일 5서 평균이게,
불목인출::,
중니튼 원교은가,
정끄 효외 어도가서 짜속,
분키가,
쫌끼.
얇 불음죠?
문학꾜 턱 트가더  으약,

일시 엔쯤

참 근데요.
그니까.
그러면 보신싼잖아.
약간 것도 박 필요.
진짜?
되고.
벤척이든 두도 연고 !클지타도 말아

아::게 뭐~ 되게 거든요?
응.
그래서 그런 거 같애요.
그렇게 거 같은 거 있어요.
아::,
백칠 뭐~,
삼속 수도 분포를 따르면 되게 되는 거지.
헤 이렇게 되는 거 같아요,
그러면은,
이런 것도 있고,
이거 있

--- 다양성 =  1.0
--- 시드 = ",
누굴 막냐고,
응.
누구 세금 받"
,
누굴 막냐고,
응.
누구 세금 받고 지금?
어띃게 싶투에 쓰나다 되예요.
어쩔상가 것잖아.
근데 우는 수을 안 있겠는데?
이와서,
그러니까지?
큰::래,
비 네 읽고의 회퓨 힘서,
그래서 따순 막~,
언어야.
공간 가 시간오가,
그랬치는 깜금 놀라면요,
어떻게 애 이다.
아니지구,
아무지?
그럼 다섯가 말 나면,
특르,
경굴,
근데,

합 용단 이지?
근데 결속에서고왔해서 됐런 거야.
얼마 직래 삐형 놈두.
근데 뭐~ 좋은 거를 또 보다는,
정트을 직의 더 내생인서로 그렇겠지?

근데 막~ 인제 이런 거고,
이케 경다.
응.
수학피 이런 거 같애요.
유 을 함려? 거 점고,
그렇게 뒤야 싶은 거 돼요.
이거 왜 근데 이뜨에 이런 없니가,
음.
바가 사생으로,
결칠십쪽 공대는 애들은 갔다 들어를 있이야,
너구 박화면 나뼈는 계런 애들은 빌학

--- 다양성 =  1.2
--- 시드 = ",
누굴 막냐고,
응.
누구 세금 받"
,
누굴 막냐고,
응.
누구 세금 받은 거고,
얘가,
남았는 애범이다,
남기0,
일순셍,
진짜 그나사의,
큰나이 공간테 보면 요애들도 상마이 근데 아.
고 명의 밖학 하눠 위생 많고 있수고,
그렇지가,
뭐났나?
예 오쯤 다든요,
복네 추 시단,
가는 엑스무예,
한~ 핸수.
이케,
긍.
태린 생션고 만로요는 넌러피구지?
옛면고 와부.
껌통데 내예같고데.
백저병가 이 육거든,
시통이리있쪽 만론 해문저거내메요.
가기생은 있습지고,
아::~ 먼으메,
냉트면은,
슈기서 되겠 소우,
표본이 우리지 대을 고해요 뒀는데,
내가랑 헤러면오라 가야.
그런 걸 있잖아요
그.
작으지?
마별와,
팔.
같은 거 꺼든.
그런적방인원,
쎄이책 줄냐?
왜,
옛와가 덮었

그런 거 같애요.
그래서 이렇게 보면 되게

--- 다양성 =  1.0
--- 시드 = "겠지?
너가 만약에 뭐~,
문과 쪽으"
겠지?
너가 만약에 뭐~,
문과 쪽으로 나눠 벌었고,
어평균이 이런 과엔 그랬나라고만 자년이건 뭐다?
내 너무 막~ 막~ 인제,
당국에도 해 하던데,
태근데 평균하다 말었는데,
픽그래라 나눠를 계고올하다 되는데.
필망외 그런 애들 좋은데,
싸::를 평균이,
경기지?
에,
가 머리지구 인제,
왜 백칠로하기고,
근데 되게 다섯가는 것도 왔는,
그러면 필,
아0.
응.
비약간,
이사이래서 뭔형?
그렇게 무섭데지가 있으로.
일본을 평균.

불
점은 이게 있다 그래서 그러면요?
그지?
삼 들양데 들어 오형에,
응.
백케 추는 커는 것도 오러면,
우리는 거 되잖아,
재뢰구,
싸기대은 그니까,
엄치::.
쎄뭔기 불틀어X하는 하면 않실에.
너무 수차는 말 반사하고,
수산다고 그렇게,
솔직야,
알박가는데,
비어두로 계구,
우리면 인제 진.
일 그니까 귀신 닦

--- 다양성 =  1.2
--- 시드 = "겠지?
너가 만약에 뭐~,
문과 쪽으"
겠지?
너가 만약에 뭐~,
문과 쪽으로점 한 잘 데번,
를를 생직에,
이 있어요?애:,
양쩔 포라 총해,
가 했예요.
솔무만 안 내와만,
그래서 좋은 자고 싸학년이의나?
일계고서만,
비파에냐,
에세 자기 표준져도황.
성배
이.
픽

혼자,
삼음.
김0픽차,
별이고 7물츠미-.
(빠마!에와수를십 평균이래 
씨놈원 용 면도애들과 들이라고,
얘예학짜.
우변서 일모 여장장.
은드똑은 힘교줘도 탐도 뭐리고,
분산은 파운하자요.
이거야?
피일 이~ 지남해서 인제요?
응.
뭐~ 정건 그러면 힘시해요.
그런 게요.
진짜 정규 닦해고 에요?
그~ 뭐,
언정으다.
옆간 거가 뭐~ 뭐~,
한두찌.
쪼레 스트 죠:::: 대개 쳤엑스터아같은,
표준:데,
또.
정말 안 목장하고,
세 쫌체들이있,
이쪽큼 안생던데,
뭐니깐
되게 평:::::짜 손재같은 요금한 경서 없거 

-------------------------------------

올가 대리 아 먹고 힘들할 가도 좀든요?
햄겠다라 되면 되게 오음같,
자기 만관고 되겠지?
그렇지라고,
곱네 한테 나형어요.

적테 니구?
아니,
이게?
되게,
어.
뭐~ 찾기가 절거 보고 그런 좀 또,
그 담하십 점생이 때 보기 아요?
찬는 쪽인 주돗으만 입리구는 같아 가 잖아요.
네.
학한 분산.
아 성.
영학대 많이 없었네.
그런 애들이 따양 출성.
픽.
그럼라 였식이,
남자기서한 끝님색,
분산이 줄교.
특만 소리 싶다 가 알갱이니까요구,
응.
알지.
지음에다,
내 얼마기 합안단요?
적뜨 뭐려더 시해죠가 가 봐 나는:짜요::무퇴이래서 던데?
그런 거 보다음
뭐 공대는 건을 건다니까,
액 오형이 해서.
지녀,
네.
그니까,
이렇게 말야.
그냥 일실어요:::짜 좋다 선한 하고 가도 들고 있잖요,
또,
차을 

--- 다양성 =  1.2
--- 시드 = ".
재수 되게 많을 수도 있다고.
올"
.
재수 되게 많을 수도 있다고.
올었나라 색 이야,
늘신 팔.
아무,
XX책서,
돈의 했나고 크요.
어디예서,
발스트센워?
막~ 응.
먼회만에요.

그래서 불학할 오고,
그럼 어디죠다 뚜센는데,
엑스끼?
이 검은 몫정도.
아청?
뭐~ 영뭐야서,
하게 무사준까구처,
손얘  규 사람들이율?
핸국 막~ 엑스다.
도 년이야.
한머니 하른 될 준라고.
옆할 위은 줄 때려 안 싫어요요?
공희가는 사람위 사궜메요,
또 귀븐 둘의 매죠이에요,
이러믄 평균,
곱:: 소리 서 없단 먹고나해야.
그래서
되분에.
요차년 해가고에,
감퍼.
네.
막금.
그것도 따르다,
기자 주에 가지.
하가 다 단가 인제,
네.
저게?
뭐는:게,
표준 전에도 때수 좀 고 안 싫어.
크기 시에에 될안 고울꺼,


어.
하준,
응.
구어?
맞어요.
어떻게 싶어서랜?
예.
다섯가지?
키


In [23]:
a = np.array([[1,2,3],[0,1,2],[3,0,1]])
a

array([[1, 2, 3],
       [0, 1, 2],
       [3, 0, 1]])

In [24]:
b = np.array([[2,0,1],[0,1,2],[1,0,2]])
b

array([[2, 0, 1],
       [0, 1, 2],
       [1, 0, 2]])

In [29]:
sum(sum(a*b))

15

In [26]:
np.matmul(a,b)

array([[ 5,  2, 11],
       [ 2,  1,  6],
       [ 7,  0,  5]])

### 합성곱 신경망

In [31]:
mnist = input_data.read_data_sets("./MNIST", one_hot=True)

Extracting ./MNIST/train-images-idx3-ubyte.gz
Extracting ./MNIST/train-labels-idx1-ubyte.gz
Extracting ./MNIST/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/t10k-labels-idx1-ubyte.gz


In [32]:
pixels = 28*28
nums = 10
#이미지 데이터 입력
x_data = tf.placeholder(tf.float32, shape=(None, pixels), name="x_data")
#정답 레이블 입력
y_data = tf.placeholder(tf.float32, shape=(None, nums), name="y_data")

In [35]:
# 가중치 초기화
def weight_variable(name, shape):
    W_init = tf.truncated_normal(shape, stddev=0.1)
    W = tf.Variable(W_init, name="b_" + name)
    return W

# 편향을 초기화
def bias_variable(name, size):
    b_unit = tf.constant(0.1, shape=[size])
    b = tf.Variable(b_unit, name="b_" + name)
    return b

# 합성곱 계층 생성
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# 최대 풀링을 생성하여 합성곱 계층 다음에 풀링을 넣는게 일반적
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [38]:
# 합성곱 1층 conv1 지정
with tf.name_scope('conv1') as scope:
    #5x5 필터, 입력 채널1,  출력 채널 32 (입력이 1인 이유는 흑백 이미지이기 때문)
    W_conv1 = weight_variable('conv1', [5, 5, 1, 32]) 
    b_conv1 = bias_variable('conv1', 32)
    x_image = tf.reshape(x_data, [-1, 28, 28, 1])

    # relu 활성화 함수를 수행한다.
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [39]:
# 최대 풀링 처리
with tf.name_scope('pool1') as scope:
    h_pool1 = max_pool(h_conv1)

In [40]:
# 합성곱 2층 conv2 지정
with tf.name_scope('conv2') as scope:
    #5x5 필터, 입력 채널32, 출력 채널64
    W_conv2 = weight_variable('conv2', [5,5,32,64])
    b_conv2 = bias_variable('conv2', 64)
    
    # relu 활성화 함수를 수행한다.
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

In [41]:
# 풀링 2층
with tf.name_scope('pool2') as scope:
    h_pool2 = max_pool(h_conv2)

In [42]:
# 전결합층를 지정하며 전결합층은 합성곱층과 풀링층의 결과인 2차원 특징 맵을 1차원으로 전개하는 역할을 함
with tf.name_scope('fully_connected') as scope:
    # 전결합층에 이르기까지 2x2의 풀링을 두 번 통과하므로 이미지의 크기는 28/2/2가 되어 7x7이 됨
    n = 7 * 7 * 64
    W_fc = weight_variable('fc', [n, 1024])
    b_fc = bias_variable('fc', 1024)
    h_pool2_flat = tf.reshape(h_pool2, [-1, n])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc) + b_fc)

In [43]:
#과잉 적합을 피하기 위해 드롭아웃 처리
with tf.name_scope('dropout') as scope: 
    keep_prob = tf.placeholder(tf.float32) 
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)

In [44]:
# 출력층을 생성
with tf.name_scope('readout') as scope:
    W_fc2 = weight_variable('fc2', [1024, 10])
    b_fc2 = bias_variable('fc2', 10)
    y_conv = tf.nn.softmax(tf.matmul(h_fc_drop, W_fc2) + b_fc2)
with tf.name_scope('loss') as scope:
    cross_entoropy = -tf.reduce_sum(y_data * tf.log(y_conv))
with tf.name_scope('training') as scope:
    optimizer = tf.train.AdamOptimizer(1e-4) 
    train_step = optimizer.minimize(cross_entoropy)
with tf.name_scope('predict') as scope:
    predict_step = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_data, 1)) 
    accuracy_step = tf.reduce_mean(tf.cast(predict_step, tf.float32))

In [45]:
def set_feed(images, labels, prob):
    return {x_data: images, y_data: labels, keep_prob: prob}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tw = tf.summary.FileWriter('/Users/lesson6_mac/tflog', graph=sess.graph)
    test_fd = set_feed(mnist.test.images, mnist.test.labels, 1)
    for step in range(10000):
        batch = mnist.train.next_batch(50)
        fd = set_feed(batch[0], batch[1], 0.5)
        loss = sess.run([train_step, cross_entoropy], feed_dict=fd)
        if step % 100 ==0:
            acc = sess.run(accuracy_step, feed_dict=test_fd)
            print("step= ", step, "loss= ", loss, "acc= ", acc)
    acc = sess.run(accuracy_step, feed_dict=test_fd)
    print("정확도 = ", acc)

step=  0 loss=  [None, 408.8066] acc=  0.081
step=  100 loss=  [None, 55.62236] acc=  0.8541
step=  200 loss=  [None, 23.801367] acc=  0.9108
step=  300 loss=  [None, 19.184723] acc=  0.9281
step=  400 loss=  [None, 21.335497] acc=  0.943
step=  500 loss=  [None, 8.949362] acc=  0.9453
step=  600 loss=  [None, 9.617835] acc=  0.9525
step=  700 loss=  [None, 6.727031] acc=  0.9558
step=  800 loss=  [None, 10.841656] acc=  0.9588
step=  900 loss=  [None, 3.6663675] acc=  0.9608
step=  1000 loss=  [None, 8.358668] acc=  0.9637
step=  1100 loss=  [None, 18.603931] acc=  0.9651
step=  1200 loss=  [None, 2.6212873] acc=  0.9676
step=  1300 loss=  [None, 12.929735] acc=  0.9704
step=  1400 loss=  [None, 21.652164] acc=  0.9715
step=  1500 loss=  [None, 6.4530754] acc=  0.97
step=  1600 loss=  [None, 5.333706] acc=  0.9743
step=  1700 loss=  [None, 8.129166] acc=  0.9748
step=  1800 loss=  [None, 6.0157595] acc=  0.9757
step=  1900 loss=  [None, 3.9743493] acc=  0.9776
step=  2000 loss=  [None